#### Mattie Gisselbeck (gisse015@umn.edu)
#### Date Created: November 16, 2022
#### Date Last Motified: November 16, 2022
#### Description: Lab 3

### Part 2.1
ETL Pipeline for Extracting Weather Data from NDAWN (www.ndawn.ndsu.nodak.edu)

In [4]:
# Part 2.1.1: Import Packages 

import arcpy
import pandas as pd
import requests

In [7]:
# Part 2.1.2: Weather Data - Last 30 Days (ndawn.ndsu.nodak.edu)

url = "https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=174&station=142&station=138&station=161&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=172&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=152&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddmxt&variable=ddmnt&variable=ddavt&year=2022&ttype=daily&quick_pick=30_d&begin_date=2022-11-26&end_date=2022-11-26"

In [11]:
# Part 2.1.3: Request Weather Data 

## Request Weather Data as a .csv 
response = requests.get(url)

## Extract Weather Data into a .csv
file_name = r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab3\last_30_days.csv'
csv = open(file_name,'w')
csv.write(response.text)
csv.close()

## Extract CSV and Read into Dataframe
raw_df = pd.read_csv(file_name, header=3, skiprows=[4])
raw_df.rename(columns={'Unnamed: 0':'Station Name', 'deg':'Lat', 'deg.1':'Lon', 'Degrees F':'Max', 'Degrees F.1':'Min', 'Degrees F.2':'Avg'}, inplace=True)
raw_df.head()

,Station Name,Lat,Lon,ft,Unnamed: 4,Unnamed: 5,Unnamed: 6,Max,Unnamed: 8,Min,Unnamed: 10,Avg,Unnamed: 12
0,Ada,47.32119,-96.51406,910,2022,10,29,60.584,NaN,30.619,NaN,45.602,NaN
1,Ada,47.32119,-96.51406,910,2022,10,30,58.784,NaN,22.091,NaN,40.438,NaN
2,Ada,47.32119,-96.51406,910,2022,10,31,64.022,NaN,35.074,NaN,49.548,NaN
3,Ada,47.32119,-96.51406,910,2022,11,1,70.610,NaN,27.538,NaN,49.074,NaN
4,Ada,47.32119,-96.51406,910,2022,11,2,68.216,NaN,51.998,NaN,60.107,NaN


In [14]:
# Create Copy of Data with Relevant Columns
columns = ['Station Name', 'Lat', 'Lon', 'Max', 'Min', 'Avg']

cleaned_df = raw_df[columns].copy()

# Aggregate Data
agg_functions = {'Lat':'first', 'Lon':'first', 'Max':'mean', 'Min':'mean', 'Avg':'mean'}
agg_df = cleaned_df.groupby(cleaned_df['Station Name']).aggregate(agg_functions)

agg_df.head()

,Lat,Lon,Max,Min,Avg
Station Name,,,,,
Ada,47.321190,-96.514060,37.143000,20.608200,28.875800
Adams,48.499880,-98.075880,33.255900,16.112300,24.684367
Alamo,48.546520,-103.471860,32.186133,12.054900,22.120733
Alvarado,48.245942,-97.021532,35.760833,18.489367,27.125367
Amidon,46.488440,-103.316290,36.632433,16.132867,26.382900


In [16]:
# Export Aggregated Dataframe to CSV

agg_df.to_csv(r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab3\aggregated_temps.csv')

In [17]:
# Converting Weather Data to a Feature Class

csv_path = r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab3\aggregated_temps.csv'

temperature_features = arcpy.management.XYTableToPoint(csv_path, 'station_temperatures', 'Lon', 'Lat')

### Part 2.2: Comparing Interpolation Methods
Examining IDW, GPI, and Kriging using 'Last 30 Days' Weather from NDAWN  

In [20]:
# Part 2.2.1: Inverse Distance Weighting (IDW) 

arcpy.ddd.Idw("station_temperatures", "Avg", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\IDW", 0.0144790399999999, 2, "VARIABLE 141", None)

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab3\\Lab3.gdb\\IDW'>

In [20]:
# Part 2.2.2: Kriging
## Kriging Method: Ordinary - Semi-variogram Model: Circular

arcpy.sa.Kriging("station_temperatures", "Avg", "Circular 0.014479 # # #", 0.0144790399999999, "VARIABLE 141", None, r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\kriging")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab3\\Lab3.gdb\\IDW'>

In [ ]:
# Part 2.2.3: Global Polynomial Interpolation (GPI)

arcpy.ga.GlobalPolynomialInterpolation("station_temperatures", "Avg", "gpi_layer", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\gpi_raster", 0.0144790399999999, 1, None)